# Download Data

In [1]:
%matplotlib widget
import matplotlib.pylab as plt
import seaborn as sns

import numpy as np

import pandas as pd

import re

from tqdm import tqdm
import glob
import os

from owslib.wms import WebMapService
import rasterio

## Floading maps

For Denmark: https://gis.nst.dk/server/rest/services/ekstern

The newest ones (Plantrin2) are divided in 14 zones.

In [2]:
df_flood_maps = pd.read_csv('Plantrin_2.csv')
df_flood_maps.head(20)

,zone,url
0,Aabenraa,https://gis.nst.dk/server/services/ekstern/Aab...
1,Esbjerg,https://gis.nst.dk/server/services/ekstern/Esb...
2,Fredericia,https://gis.nst.dk/server/services/ekstern/Fre...
3,Holstebro,https://gis.nst.dk/server/services/ekstern/Hol...
4,Juelsminde,https://gis.nst.dk/server/services/ekstern/Jue...
5,Kobenhavn_NS,https://gis.nst.dk/server/services/ekstern/Kob...
6,Kobenhavn_Nord,https://gis.nst.dk/server/services/ekstern/Koe...
7,Kobenhavn_Syd,https://gis.nst.dk/server/services/ekstern/Koe...
8,Kolding,https://gis.nst.dk/server/services/ekstern/Kol...
9,Korsoer,https://gis.nst.dk/server/services/ekstern/Kor...


In [3]:
# Quick fix. These two are not in WMS format. Needs to be fixed later
df_flood_maps.drop([6,7], inplace=True);

## Check what layers are available

In [4]:
wms = WebMapService(df_flood_maps.iloc[3].url+'/WmsServer')

for layer_nb in wms.contents:
    print('%s %40s %s'%(layer_nb,wms[layer_nb].title,wms[layer_nb].crsOptions))

0                             Risikoområde ['EPSG:4326', 'EPSG:25832']
3 Kulturarv_Seværdige_Fortidsminder_punkter ['EPSG:4326', 'EPSG:25832']
4  Kulturarv_Seværdige_Fortidsminder_areal ['EPSG:4326', 'EPSG:25832']
5                         Kulturarv_Kirker ['EPSG:4326', 'EPSG:25832']
6              Kulturarv_Fredede_Bygninger ['EPSG:4326', 'EPSG:25832']
7          Kulturarv_Fortidsminder_punkter ['EPSG:4326', 'EPSG:25832']
8            Kulturarv_Fortidsminder_areal ['EPSG:4326', 'EPSG:25832']
9                       Bes_sten_jorddiger ['EPSG:4326', 'EPSG:25832']
10                       Begravelsesomraade ['EPSG:4326', 'EPSG:25832']
12                         Vandvaerksboring ['EPSG:4326', 'EPSG:25832']
13    Miljoeinteresser_Vandforsyningsboring ['EPSG:4326', 'EPSG:25832']
14              Miljoeinteresser_Natura2000 ['EPSG:4326', 'EPSG:25832']
15         Miljoeinteresser_Beskyttet_natur ['EPSG:4326', 'EPSG:25832']
16    Miljoeinteresser_Aktiv_vandindvinding ['EPSG:4326', 'EPSG:25832']

In [5]:
wms = WebMapService(df_flood_maps.iloc[0].url+'/WmsServer')

for layer_nb in wms.contents:
    print('%s %40s %s'%(layer_nb,wms[layer_nb].title,wms[layer_nb].crsOptions))

0                             Risikoområde ['EPSG:4326', 'EPSG:25832']
3 Kulturarv_Seværdige_Fortidsminder_punkter ['EPSG:4326', 'EPSG:25832']
4  Kulturarv_Seværdige_Fortidsminder_areal ['EPSG:4326', 'EPSG:25832']
5                         Kulturarv_Kirker ['EPSG:4326', 'EPSG:25832']
6              Kulturarv_Fredede_Bygninger ['EPSG:4326', 'EPSG:25832']
7          Kulturarv_Fortidsminder_punkter ['EPSG:4326', 'EPSG:25832']
8            Kulturarv_Fortidsminder_areal ['EPSG:4326', 'EPSG:25832']
9                       Bes_sten_jorddiger ['EPSG:4326', 'EPSG:25832']
10                       Begravelsesomraade ['EPSG:4326', 'EPSG:25832']
12                         Vandvaerksboring ['EPSG:4326', 'EPSG:25832']
13    Miljoeinteresser_Vandforsyningsboring ['EPSG:4326', 'EPSG:25832']
14         Miljoeinteresser_Beskyttet_natur ['EPSG:4326', 'EPSG:25832']
15    Miljoeinteresser_Aktiv_vandindvinding ['EPSG:4326', 'EPSG:25832']
17                       Risikovirksomheder ['EPSG:4326', 'EPSG:25832']

We'll be interested in the *Udbredelse* layers for the diferent scenarios:

    Udbredelse_AAB_MT20_2019
    Udbredelse_AAB_MT100_2115
    Udbredelse_AAB_MT100_2065
    Udbredelse_AAB_MTEXT_2019
    Udbredelse_AAB_MTEXT_2115
    Udbredelse_AAB_MT100_2019
    
Since they can be in different layer numbers for the different zones, we'll check for each one

In [5]:
def check_layer_number(url,layer_name):
    wms = WebMapService(url+'/WmsServer')
    layer_titles = [wms[layer_nb].title for layer_nb in wms.contents]
    layer_number = [layer_nb for layer_nb in wms.contents if re.match(layer_name,wms[layer_nb].title) ]
    return layer_number

In [6]:
scenarios = ['MTEXT_2115',
             'MTEXT_2019',
             'MT100_2115',
             'MT100_2065',
             'MT100_2019',
             'MT20_2019',
             'MT50_2115',
             'MT50_2065',
             'MT50_2019',
            ]

for scenario in scenarios:
    df_flood_maps['Udbredelse_'+scenario] = df_flood_maps.url.apply(lambda x: check_layer_number(x,'Udbredelse_\w+_'+scenario))

In [7]:
df_flood_maps.head()

,zone,url,Udbredelse_MTEXT_2115,Udbredelse_MTEXT_2019,Udbredelse_MT100_2115,Udbredelse_MT100_2065,Udbredelse_MT100_2019,Udbredelse_MT20_2019,Udbredelse_MT50_2115,Udbredelse_MT50_2065,Udbredelse_MT50_2019
0,Aabenraa,https://gis.nst.dk/server/services/ekstern/Aab...,[94],[97],[95],[96],[98],[99],[],[],[]
1,Esbjerg,https://gis.nst.dk/server/services/ekstern/Esb...,[96],[99],[97],[98],[100],[101],[],[],[]
2,Fredericia,https://gis.nst.dk/server/services/ekstern/Fre...,[91],[94],[92],[93],[95],[96],[],[],[]
3,Holstebro,https://gis.nst.dk/server/services/ekstern/Hol...,[],[],[96],[],[99],[101],[97],[98],[100]
4,Juelsminde,https://gis.nst.dk/server/services/ekstern/Jue...,[94],[97],[95],[96],[98],[99],[],[],[]


## Download data and save it with georeferences

In [8]:
def save_file(url,layer,size,crs,dir_path):
    
    wms = WebMapService(url+'/WmsServer')
    srs = wms[layer].crsOptions
    bbox = wms[layer].boundingBoxWGS84 
    output_name = wms[layer].title
    
    if crs not in srs:
        print('CRS not available (available options:)',srs)
        pass
        
    # Get the image from the server
    img = wms.getmap(layers=[layer],
                     styles=['default'],
                     srs=crs,
                     bbox=bbox,
                     size=size,
                     transparent=True,
                     format='image/png8'
                    )
    
    # Save the array as an image
    print('Saved', dir_path+output_name+'.png')
    with open(dir_path+output_name+'.png', 'wb') as out:
        out.write(img.read())
       
    # Re-open the same image in rasterio to georeference it
    img_array = rasterio.open(dir_path+output_name+'.png').read().squeeze() 
    img_shape = img_array.shape
    dst_transform = rasterio.transform.from_bounds(west = bbox[0],
                                                   south = bbox[1],
                                                   east = bbox[2],
                                                   north = bbox[3],
                                                   width=img_shape[1], height=img_shape[0])
    # Write it out to a file.
    with rasterio.open(dir_path+output_name+'.png', 'w', driver='GTiff',
                                  width=img_shape[1], height=img_shape[0],
                                  count=1, dtype=np.uint8, nodata=0,
                                  transform=dst_transform, crs=crs) as dst:
        dst.write(img_array.astype(np.uint8), indexes=1)

In [13]:
for scenario in tqdm(scenarios):
    
    dir_path = 'Udbredelse_'+scenario+'/'
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

    for idx,row in df_flood_maps.iterrows():
        if len(row['Udbredelse_'+scenario]) == 0:
            print('Scenario for ',row['Udbredelse_'+scenario],'does not exist')
            continue
        else:
            # there can be more than one layer (eg. cph north and south)
            for layer in row['Udbredelse_'+scenario]:
                save_file(row['url'],
                          layer=layer,
                          size=(1000,1000),
                          crs='EPSG:4326',
                          dir_path=dir_path)

  0%|          | 0/1 [00:00<?, ?it/s]

Saved Udbredelse_MTEXT_2019/Udbredelse_AAB_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_ESB_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_FRE_MTEXT_2019.png
Scenario for  [] does not exist
Saved Udbredelse_MTEXT_2019/Udbredelse_JUE_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_KHS_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_KHN_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_KOL_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_KOR_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_KBU_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_NYB_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_ODE_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_RAF_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_SKB_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_SYD_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_NAK_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_VEJ_MTEXT_2019.png


100%|██████████| 1/1 [00:30<00:00, 30.50s/it]

Saved Udbredelse_MTEXT_2019/Udbredelse_VOR_MTEXT_2019.png
